## env id, paths, installs, imports

In [1]:
# Function to check env (colab or local)
import sys

def check_environment():
    if 'google.colab' in sys.modules:
        # Running in Google Colab
        return "Google Colab"

    if hasattr(sys, 'prefix'):
        if sys.prefix.startswith('/usr/local'):
            # Running in a Colab-like environment (could be Colab or another cloud environment)
            return "Colab-like environment"
        else:
            # Running in a local environment
            return "Local environment"

    # Default to local environment if checks are inconclusive
    return "Local environment"

In [2]:
# set paths and install packages depending on the environment

if check_environment() == 'Google Colab':
    # !pip install transformers torch bertopic openai natasha --quiet
    !pip install transformers torch bertopic --quiet
    from google.colab import drive
    drive.mount('/content/drive')
    data_path = '/content/drive/MyDrive/Colab Notebooks/Narratives/notebooks_clustering(topics)/clustering4Katya/'
    result_path = '/content/drive/MyDrive/Colab Notebooks/Narratives/notebooks_clustering(topics)/clustering4Katya/results/'

else:
    data_path = '/Users/alexbadin/Library/CloudStorage/GoogleDrive-alex.badin@gmail.com/My Drive/Colab Notebooks/Narratives/notebooks_clustering(topics)/clustering4Katya/'
    result_path = '/Users/alexbadin/Library/CloudStorage/GoogleDrive-alex.badin@gmail.com/My Drive/Colab Notebooks/Narratives/notebooks_clustering(topics)/clustering4Katya/results/'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 62.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 

In [3]:
import pandas as pd
import json

# BERTopic imports
from bertopic import BERTopic
from hdbscan import HDBSCAN
from bertopic.representation import KeyBERTInspired
from umap import UMAP

In [4]:
file_path = data_path+'truestory_sample.json'
# open as json
with open(file_path) as f:
    data = json.load(f)
len(data[0])

792

In [7]:
data[0][:2]

['В поддержку предложенной Иорданией и еще 40 странами резолюции выступили 120 государств.\n\n© Фото: Lev Radin Keystone Press Agency Globallookpress\n\nВ Нью-Йорке Генеральная Ассамблея ООН приняла резолюцию по палестино-израильскому конфликту, предложенную группой арабских стран. Резолюция, которую подготовили Иордания и еще четыре десятка стран, призывает к немедленному перемирию. «За» проголосовали 120 государств, «Против» - 14 и еще 45 участников ГА ООН воздержались от голосования. Об этом сообщает РИА Новости.\n\nПо информации агентства, принятая резолюция осуждает направленное на мирных граждан Израиля и Палестины насилие, призывает к немедленному гуманитарному перемирию, которое бы привело к прекращению боевых действий. Помимо этого, в принятом документе звучит призыв к освобождению гражданских лиц, «незаконно удерживаемых в плену», немедленно и без предварительных условий.\n\nТакже было обозначено требование незамедлительно и без препятствий доставить в сектор Газа необходимые

# Clustering

In [8]:
# function to calculate clusters and probs for a given texts
def calc_clusters(texts):
    umap_model = UMAP(n_neighbors=15, n_components=10, metric='cosine', low_memory=False, random_state=42)
    representation_model = KeyBERTInspired()
    # run BERTopic clustering
    topic_model = BERTopic(language="russian", umap_model=umap_model, nr_topics = 'auto',\
                        calculate_probabilities=False, min_topic_size = 20
                        # , representation_model = representation_model
                        , embedding_model = 'intfloat/multilingual-e5-large' #- KeyBERT requires embed original model
                        )
    topics, prob = topic_model.fit_transform(texts)
    return topics, prob

for each group of texts in json:
- create df with column 'text'
- deduplicate
- remove NaN
- calc clusters with probs
- save clusters & prob in respective columns
- save as csv

In [ ]:
from tqdm import tqdm
# iterate through json
for i in tqdm(range(len(data))):
    # create df with column 'text'
    df = pd.DataFrame(data[i], columns=['text'])
    # deduplicate
    df = df.drop_duplicates()
    # remove NaN
    df = df.dropna()
    # calc clusters with probs
    df['cluster'], df['prob'] = calc_clusters(df['text'])
    # save as csv
    df.to_csv(result_path+f'clustered_texts_{i}.csv', index=False)

100%|██████████| 28/28 [17:07<00:00, 36.70s/it]
